<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/1_EDA_and_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predicting UVI with LSTMs

[Notebook 1: EDA and Cleaning](./1_EDA and Cleaning.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling and Predictions.ipynb)

[Notebook 3: Technical Report](./3_Technical_Report.ipynb)

This notebook explores raw UVI-data from the BTS2048-UV-WP spectroradiometer and prepares it for modeling.


- Read in UVI-data
- Negative values and unnecessary features
- Plot basic relationships
- EDA (exploratory data analysis)
- cleaning

In [ ]:
## import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

import matplotlib
import seaborn as sns

# Import der Messdaten

Die Messdaten sind in einer CSV-Datei gespeichert, diese muss importiert werden.

In [ ]:
## Code zum Import der Messdaten

# Bereinigen der Messdaten

Hier wird erklärt, was zum Bereinigen der Messdaten getan werden muss.

- Fehlende Messtage müssen ersetzt werden
- Outliers müssen ersetzt werden

In [ ]:
## Code zum Bereinigen der Messdaten

# Plotten von Zusammenhängen in den Messdaten

In [ ]:
## Code zum Plotten von Zusammenhängen in den Messdaten

# Daten zu "pickel" speichern

https://docs.python.org/3/library/pickle.html

 - The pickle module implements binary protocols for serializing and de-serializing a *Python object structure*. “Pickling” is the process whereby a *Python object hierarchy is converted into a byte stream*, and “unpickling” is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into an object hierarchy. Pickling (and unpickling) is alternatively known as “serialization”, “marshalling,” 1 or “flattening”; however, to avoid confusion, the terms used here are “pickling” and “unpickling".

In [ ]:
## Code um Messdaten zu pickel zu speichern